In [4]:
import pandas as pd
import numpy as np
import requests
import json

API_URL = "http://127.0.0.1:8000/predict"
n_samples = 5

data = pd.read_csv("../data/hotel_bookings.csv") #Utilizar el path en donde se ubica hotel_booking.csv
var_objective = ['is_canceled']

data['arrival_date'] = pd.to_datetime(data['arrival_date_year'].astype(str) + '-' + data['arrival_date_month'].astype(str) + '-' + data['arrival_date_day_of_month'].astype(str))
data['day_of_week'] = data['arrival_date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)

sample = data.dropna().sample(n=n_samples)

y_norm = sample[var_objective]
sample = sample.drop(var_objective, axis=1)

sample = sample.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)
sample = sample.reindex(columns=sample.columns)
sample = sample.where(pd.notnull(sample), None)
sample = sample.apply(lambda s: s.map(lambda v: v.item() if isinstance(v, np.generic) else v))
payload = {"data": sample.to_dict(orient="records")}

try:
    r = requests.post(API_URL, json=payload, timeout=30)
    r.raise_for_status()
    resp = r.json()
except requests.exceptions.RequestException as e:
    print("\nError al llamar a la API:", e)
    raise

if "predictions" in resp:
    preds = pd.DataFrame({"prediction": resp["predictions"]})
    print("\nPredicciones:")
    print(preds)
    print("\nValores esperados:")
    print(y_norm)
else:
    print("\nRespuesta sin clave 'predictions':")
    print(resp)


Predicciones:
   prediction
0           0
1           0
2           1
3           0
4           0

Valores esperados:
       is_canceled
18263            0
18279            0
2971             1
18313            0
2900             0


C:\Users\jalva\AppData\Local\Temp\ipykernel_21304\4226659313.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sample = sample.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)
